In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/natam-tweets-oct2022/native_tweets_Oct2022.csv


In [5]:
# Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter

# Date and time
import datetime as dt
from datetime import date, datetime, timedelta

# Geography
from geopy.geocoders import Nominatim
from functools import partial

# Text, NLP
from spacy.language import Language
import spacy
import gensim
import numerizer
import demoji
import contractions
import re
import string
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# Visualizations
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis
import pyLDAvis.gensim
from tabulate import tabulate

/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """


In [6]:
# Import dataset
data = pd.read_csv('../input/natam-tweets-oct2022/native_tweets_Oct2022.csv')

# Check
data.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,search_term
0,1584221836828516352,1584221836828516352,1.666543e+12,2022-10-23 11:34:53,-500,NaN,Activist Sacheen Littlefeather exposed by sist...,en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
1,1584221758025973760,1583978820830654465,1.666543e+12,2022-10-23 11:34:35,-500,NaN,@vincescarlet @nasescobar316 The point is you ...,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'vincescarlet', 'name': 'Dani...",NaN,NaN,NaN,NaN,Native American
2,1584221636202729472,1584221636202729472,1.666543e+12,2022-10-23 11:34:06,-500,NaN,https://t.co/GulvGiohqi,zxx,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
3,1584221631118835712,1584221631118835712,1.666543e+12,2022-10-23 11:34:04,-500,NaN,Complete 4 Page Apa Formatted Essay Native Ame...,en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
4,1584221608356388865,1584221608356388865,1.666543e+12,2022-10-23 11:33:59,-500,NaN,"Speaking From Orbit, the First Native-American...",en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American


# Data Cleaning
- Find and remove columns that are totally empty.
- Find and remove columns that have no variability in responses (all cells have the same value).
- Remove columns that add no value to the analysis.

In [7]:
# Find any columns that are totally empty
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2984 non-null   int64  
 1   conversation_id  2984 non-null   int64  
 2   created_at       2984 non-null   float64
 3   date             2984 non-null   object 
 4   timezone         2984 non-null   int64  
 5   place            5 non-null      object 
 6   tweet            2984 non-null   object 
 7   language         2984 non-null   object 
 8   hashtags         2984 non-null   object 
 9   cashtags         2984 non-null   object 
 10  user_id          2984 non-null   int64  
 11  user_id_str      2984 non-null   int64  
 12  username         2984 non-null   object 
 13  name             2984 non-null   object 
 14  day              2984 non-null   int64  
 15  hour             2984 non-null   int64  
 16  link             2984 non-null   object 
 17  urls          

In [8]:
# List of columns to drop
drop_col_list = []

# Add empty columns to the drop list
for col in data.columns:
    # If the column is totally empty
    if data[col].isnull().sum() == data.shape[0]:
        # Add to drop list
        drop_col_list.append(col)
    else:
        # Do nothing
        continue

# Check
print(f"{len(drop_col_list)} columns have been added to the drop column list.")

10 columns have been added to the drop column list.


In [9]:
# Check if all values are the same in a column
for col in data.columns:
    # Skip if not in drop list
    if col not in drop_col_list:
        # Check if there is no variation 
        if data[col].nunique() == 1:
            # Add to drop list
            drop_col_list.append(col)
        # else
        else:
            # Do nothing
            continue
    # Continue
    else:
        # Do nothing
        continue
        
# Check
print(f"There are {len(drop_col_list)} columns in the drop columns list.")

There are 12 columns in the drop columns list.


Columns that hold no value in the analysis include:
- id
- created_at
- date
- timezone
- user_id
- user_id_str
- link
- search_term

In [10]:
# Print drop column list
drop_col_list

['near',
 'geo',
 'source',
 'user_rt_id',
 'user_rt',
 'retweet_id',
 'retweet_date',
 'translate',
 'trans_src',
 'trans_dest',
 'timezone',
 'retweet']

In [11]:
# Extend drop_col_list
drop_col_list.extend(['id', 'created_at', 'date', 'user_id', 'user_id_str', 'link', 'search_term'])

# Check
print(f"There are {len(drop_col_list)} items in the drop column list.")

There are 19 items in the drop column list.


In [12]:
# Drop columns
df = data.drop(columns=drop_col_list, axis=1)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  2984 non-null   int64 
 1   place            5 non-null      object
 2   tweet            2984 non-null   object
 3   language         2984 non-null   object
 4   hashtags         2984 non-null   object
 5   cashtags         2984 non-null   object
 6   username         2984 non-null   object
 7   name             2984 non-null   object
 8   day              2984 non-null   int64 
 9   hour             2984 non-null   int64 
 10  urls             2984 non-null   object
 11  photos           2984 non-null   object
 12  video            2984 non-null   int64 
 13  thumbnail        910 non-null    object
 14  nlikes           2984 non-null   int64 
 15  nreplies         2984 non-null   int64 
 16  nretweets        2984 non-null   int64 
 17  quote_url        150 non-null    

## Cleaning Data

Cleaning data will occur in phases. To start, any non-English tweets will be removed.

In [13]:
# Drop if df.language != 'en'
df = df[df['language'] == 'en']

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2540 entries, 0 to 2983
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  2540 non-null   int64 
 1   place            5 non-null      object
 2   tweet            2540 non-null   object
 3   language         2540 non-null   object
 4   hashtags         2540 non-null   object
 5   cashtags         2540 non-null   object
 6   username         2540 non-null   object
 7   name             2540 non-null   object
 8   day              2540 non-null   int64 
 9   hour             2540 non-null   int64 
 10  urls             2540 non-null   object
 11  photos           2540 non-null   object
 12  video            2540 non-null   int64 
 13  thumbnail        578 non-null    object
 14  nlikes           2540 non-null   int64 
 15  nreplies         2540 non-null   int64 
 16  nretweets        2540 non-null   int64 
 17  quote_url        140 non-null    

In [14]:
# Reset index
df.reset_index(inplace=True)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            2540 non-null   int64 
 1   conversation_id  2540 non-null   int64 
 2   place            5 non-null      object
 3   tweet            2540 non-null   object
 4   language         2540 non-null   object
 5   hashtags         2540 non-null   object
 6   cashtags         2540 non-null   object
 7   username         2540 non-null   object
 8   name             2540 non-null   object
 9   day              2540 non-null   int64 
 10  hour             2540 non-null   int64 
 11  urls             2540 non-null   object
 12  photos           2540 non-null   object
 13  video            2540 non-null   int64 
 14  thumbnail        578 non-null    object
 15  nlikes           2540 non-null   int64 
 16  nreplies         2540 non-null   int64 
 17  nretweets        2540 non-null   

In [15]:
# Drop the language column
df = df.drop(columns='language', axis=1)

# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,urls,photos,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,['https://www.foxnews.com/entertainment/activi...,[],0,NaN,0,0,0,NaN,Native American,[]


### Manipulating Features: Indicators, Counts, and Categories

For certain columns, the presence of an item should be noted. Where multiple items could exist, counts should be attached. These columns include:
- hashtags: count
- cashtags: count
- urls: indicator
- photos: count
- thumbnail: indicator
- quote_url: indicator
- reply_to: count
- Conversations: category and size

#### Hashtags

In [16]:
# Iterate through the column 
# Note that this column contains strings, not lists.
def listCounter(column):
    # list
    count_list = []
    for c in column:
        # Set string to manipulate
        c_string = c
        # Replace [ and ] and ''
        c_string = re.sub(r'[\[\]\'\']', '', c_string)
        if len(c_string) > 0:
            # Count hashtags
            count_list.append(len(c_string.split(',')))
        else:
            # Count
            count_list.append(0)
    # Return
    return count_list
    
# Set new column
df['hashtag_counts'] = listCounter(df.hashtags)
    
# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,...,photos,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to,hashtag_counts
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,...,[],0,NaN,0,0,0,NaN,Native American,[],0


In [17]:
# Add to the hashtag list
def listParser(column):
    # Create list
    parsed_list = []
    for c in column:
        # Set string to manipulate
        c_string = c
        # Replace [ and ] and ''
        c_string = re.sub(r'[\[\]\'\']', '', c_string)
        # If there is a string
        if len(c_string) > 0:
            # Split the remaining string
            for i in c_string.split(','):
                # Remove spaces
                i = i.strip()
                # Append to the list
                parsed_list.append(i)
        # Else
        else:
            # Do nothing
            continue
    # Return parsed list
    return parsed_list

In [18]:
# Create the hashtag list
hashtags = listParser(df.hashtags)

# Print
print(f"There are {len(hashtags)} hashtags found in this dataset.")

There are 1301 hashtags found in this dataset.


In [19]:
# Count of hashtags
hashtag_counter = Counter(hashtags)

# View top 10
hashtag_counter.most_common(10)

[('indigenous', 74),
 ('tairp', 59),
 ('nativeamerican', 39),
 ('americanindian', 33),
 ('foxnews', 15),
 ('fba', 14),
 ('turquoisejewelry', 13),
 ('handmadejewelrysale', 13),
 ('handmadejewelry', 13),
 ('southwesternjewelry', 13)]

#### Cashtags

In [20]:
# Set a new column 
df['cashtag_counts'] = listCounter(df.cashtags)

# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,...,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to,hashtag_counts,cashtag_counts
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,...,0,NaN,0,0,0,NaN,Native American,[],0,0


In [21]:
# Create the cashtags list
cashtags = listParser(df.cashtags)

# Print
print(f"There were {len(cashtags)} cashtags in the dataset.")

There were 2 cashtags in the dataset.


In [22]:
# Drop hashtags and cashtags columns
df = df.drop(columns=['hashtags', 'cashtags'], axis=1)

#### URL Indicator

This will indicate whether a URL was included as part of the tweet.

In [23]:
# URL indicator list
url_indicator = []

# Iterate through the column
for url in df.urls:
    # URL is present
    if 'http' in url:
        url_indicator.append(1)
    # URL is not present
    else:
        url_indicator.append(0)
        
# Add to the dataframe
df['url_indicator'] = url_indicator

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            2540 non-null   int64 
 1   conversation_id  2540 non-null   int64 
 2   place            5 non-null      object
 3   tweet            2540 non-null   object
 4   username         2540 non-null   object
 5   name             2540 non-null   object
 6   day              2540 non-null   int64 
 7   hour             2540 non-null   int64 
 8   urls             2540 non-null   object
 9   photos           2540 non-null   object
 10  video            2540 non-null   int64 
 11  thumbnail        578 non-null    object
 12  nlikes           2540 non-null   int64 
 13  nreplies         2540 non-null   int64 
 14  nretweets        2540 non-null   int64 
 15  quote_url        140 non-null    object
 16  search           2540 non-null   object
 17  reply_to         2540 non-null   

In [24]:
# Collect URLs
url_list = [u for u in df.urls if 'http' in u]

# Print
print(f"There are {len(url_list)} URLs in the dataset.")

There are 765 URLs in the dataset.


In [25]:
# Check for URLs
url_counter = Counter(url_list)

# Top 10 URLs
url_counter.most_common(10)

[("['https://www.foxnews.com/entertainment/activist-sacheen-littlefeather-exposed-sisters-reported-fraudulent-native-american-identity-lie']",
  39),
 ("['https://www.sfchronicle.com/opinion/openforum/article/Sacheen-Littlefeather-oscar-Native-pretendian-17520648.php']",
  15),
 ("['https://apple.news/AjrF3y_KmSHGOoqdD_d-m1w']", 14),
 ("['https://www.breitbart.com/entertainment/2022/10/23/woke-media-fail-sacheen-littlefeathers-sisters-claim-activist-lied-about-being-native-american/']",
  11),
 ("['https://www.cnn.com/2022/10/18/health/flu-vaccination-inequity/index.html']",
  10),
 ("['https://www.sfchronicle.com/opinion/openforum/article/Sacheen-Littlefeather-oscar-Native-pretendian-17520648.php?utm_campaign=CMS%20Sharing%20Tools%20(Premium)&utm_source=t.co&utm_medium=referral']",
  9),
 ("['https://www.etsy.com/shop/Highway66Treasures?ref=seller-platform-mcnav']",
  8),
 ("['https://www.amazon.com/gp/product/B085VN2SSN/?tag=hunchpress-20/']", 7),
 ("['https://thehornnews.com/biden-q

#### Photos

In [26]:
# Photo indicator list
photo_indicator = []

# Iterate through the column
for p in df.photos:
    # If a photo is attached
    if 'http' in p:
        photo_indicator.append(1)
    # If no photo attached
    else:
        photo_indicator.append(0)
        
# Add to the dataframe
df['photo_indicator'] = photo_indicator

In [27]:
# Drop the photo column from the dataframe
df = df.drop(columns='photos', axis=1)

#### Thumbnail

In [28]:
# Thumbnail indicator list
thumbnail_indicator = []

# Iterate through the column
for t in df.thumbnail:
    # If there is no thumbnail
    if isinstance(t, float):
        thumbnail_indicator.append(0)
    # There is an indicator
    else:
        thumbnail_indicator.append(1)
        
# Add to the dataframe
df['thumbnail_indicator'] = thumbnail_indicator

In [29]:
# Drop the thumbnail column
df = df.drop(columns='thumbnail', axis=1)

#### Quote URL

In [30]:
# Quote URL indicator
qurl_indicator = []

# Iterate through the column
for q in df.quote_url:
    # No quote url
    if isinstance(q, float):
        qurl_indicator.append(0)
    # Quote URL exists
    else:
        qurl_indicator.append(1)
        
# Add to dataframe
df['qurl_indicator'] = qurl_indicator

In [31]:
# Drop the quote url column
df = df.drop(columns='quote_url', axis=1)

### Conversations

Conversations are a bit more complex. They can be categorized, as well as having a length attached to them.

Conversation categories:
- single, one tweet in a conversation ID.
- multiple, multiple tweets in a conversation ID.

In [32]:
# Get a count of the unique conversation IDs
conv_id_count = Counter(df.conversation_id)

# Conversation category
conv_cat = []

# Conversation size
conv_size = []

for cid in df.conversation_id:
    # Append to conversation size
    conv_size.append(conv_id_count.get(cid))
    # Determine category 
    if conv_id_count.get(cid) == 1:
        conv_cat.append('single')
    else:
        conv_cat.append('multiple')
        
# Add to dataframe
df['conversation_category'] = conv_cat
df['conversation_size'] = conv_size

# Drop conversation_id
df = df.drop(columns='conversation_id', axis=1)

### Day

Days can be categorized as 'weekday' or 'weekend.' Twitter scrapes return days as integers 1 thru 7, where 1 to 5 are weekdays and 6 and 7 are weekends.

In [33]:
# Day category
day_cat = []

# Iterate through the day column
for day in df.day:
    if day in range(5):
        day_cat.append('weekday')
    else:
        day_cat.append('weekend')
        
# Add to dataframe
df['day_category'] = day_cat

### Hour

Hours can be categorized according to times of day:
- AM, 00 to 11
- PM, 12 to 23

In [34]:
# Hour category
hour_cat = []

# Iterate through hour column
for hour in df.hour:
    # AM
    if hour in range(11):
        # Append
        hour_cat.append('AM')
    # PM
    else:
        hour_cat.append('PM')
        
# Hour category column
df['time_of_day'] = hour_cat

### Locations

Locations appear as a dictionary with point coordinates.

In [35]:
# Get a location index list
location_idx = df[df['place'].isna() == False].index

In [36]:
# Location indicator list
location_indicator = []

# Iterate through place list
for place in df.place:
    if isinstance(place, float):
        location_indicator.append(0)
    else:
        location_indicator.append(1)
        
# Add to the dataframe
df['loc_indicator'] = location_indicator

## Non-Tweet Text Exploratory Data Analysis

### Location

There aren't many locations provided in this dataset, but for the ones we do have, we can perform reverse geocoding via geopy to see state. This will generate sets of strings and tuples, where the information we're interested in is the string component.

In [37]:
# Set up geolocator using Nominatim service - Nominatim is free, hence the choice.
geolocator = Nominatim(user_agent="natam_twitter_research")

In [38]:
# Function to get coordinates
def getCoords(idx, df_column):
    # Get coordinates
    coordinates = (df.place[idx].split('[')[1]).split(']')[0]
    # Return 
    return coordinates

In [39]:
# Use coordinates
def getLoc(coords):
    location = geolocator.reverse(coords)
    # Based on type
    for l in location:
        # Only need the string object
        if isinstance(l,str):
            # State
            state = l.split(',')[-3]
        else:
            # Do nothing
            continue
    # Return state
    return state

In [40]:
# Hold state for location of tweet
states = []

for idx in location_idx:
    # Coordinates
    coords = getCoords(idx, df.place)
    # States
    state = getLoc(coords)
    # Append to list
    states.append(state)

In [41]:
# Counter
states_counter = Counter(states)

# Breakdown of states
state_table = tabulate({'States': states_counter.keys(),
                        'Counts': states_counter.values()},
                       headers='keys',
                       tablefmt='presto')

# Show
print(state_table)

 States     |   Counts
------------+----------
 New Mexico |        3
 Arizona    |        1
 Montana    |        1


In [42]:
# Counter
loc_ind_counter = Counter(df.loc_indicator)

# Lists of keys and values
loc_ind_viz = {'Indicator': loc_ind_counter.keys(),
               'Counts': loc_ind_counter.values()}

# Location vs. No Location
loc_pie = px.pie(loc_ind_viz, values='Counts', names=['No Location Specified', 'Location Specified'], title='Location Specified vs. No Location Specified')

# Show
loc_pie.show()

### Day and Hour

In [43]:
# Breakdown by day
day_counter = Counter(df.day)

# Keys and values
day_viz = {'Days': sorted(day_counter.keys(), reverse=False),
           'Values': [day_counter.get(1),
                     day_counter.get(2),
                     day_counter.get(3),
                     day_counter.get(4),
                     day_counter.get(5),
                     day_counter.get(6),
                     day_counter.get(7)]}

# Bar chart
day_bar = px.bar(day_viz, x=['Monday', 
                             'Tuesday', 
                             'Wednesday', 
                             'Thursday', 
                             'Friday', 
                             'Saturday', 
                             'Sunday'], y='Values', category_orders=[1,2,3,4,5,6,7], text_auto=True, title='Days Breakdown')

# Show
day_bar.show()

In [44]:
# Hours breakdown
hours_counter = Counter(df.hour)

# List of hours and counts
hours_viz = {'Hours': [0,1,2,3,4,5,6,7,8,9,10,11,
                       12,13,14,15,16,17,18,19,20,21,22,23],
             'Counts': [hours_counter.get(0), hours_counter.get(1), hours_counter.get(2), hours_counter.get(3),
                        hours_counter.get(4), hours_counter.get(5), hours_counter.get(6), hours_counter.get(7),
                        hours_counter.get(8), hours_counter.get(9), hours_counter.get(10), hours_counter.get(11),
                        hours_counter.get(12), hours_counter.get(13), hours_counter.get(14), hours_counter.get(15),
                        hours_counter.get(16), hours_counter.get(17), hours_counter.get(18), hours_counter.get(19),
                        hours_counter.get(20), hours_counter.get(21), hours_counter.get(22), hours_counter.get(23)]}

# Bar chart
hours_bar = px.bar(hours_viz, x='Hours', y='Counts', text_auto=True, title='Hours Breakdown')

# Show
hours_bar.show()

In [45]:
# AM/PM by Days
day_ampm_histo = px.histogram(df, x='day', color='time_of_day', text_auto=True, title='AM/PM by Days')

# Show
day_ampm_histo.show()

In [46]:
# Time of the week by hour
hour_day_histo = px.histogram(df, x='hour', color='day_category', text_auto=True, title='Part of the Week by the Hour')

# Show
hour_day_histo.show()

### URL Indicators

In [47]:
# URL indicators counter
url_i_counter = Counter(df.url_indicator)

# Indicator keys and values
url_i_viz = {'Indicator': url_i_counter.keys(),
             'Values': url_i_counter.values()}

# Visualization
url_pie = px.pie(url_i_viz, names=['URL Present', 'No URL Present'], values='Values', title='No URLs Present vs. URLs Present')

# Show
url_pie.show()

### Photo Indicator

In [48]:
# Photo indicators counter
photo_indicator_counter = Counter(df.photo_indicator)

# Lists to dict
photo_viz = {'Indicators': photo_indicator_counter.keys(),
             'Values': photo_indicator_counter.values()}

# Pie chart
photo_pie = px.pie(photo_viz, names=['No Photos Present', 'Photos Present'], values='Values', title="No Photos Present vs. Photos Present")

# Show
photo_pie.show()

### Thumbnail Indicator

In [49]:
# Counter
thumbnail_counter = Counter(df.thumbnail_indicator)

# List for dict
thumbnail_viz = {'Indicator': thumbnail_counter.keys(),
                 'Values': thumbnail_counter.values()}

# Pie chart
thumbnail_pie = px.pie(thumbnail_viz, names=['No Thumbnail Present', 'Thumbnail Present'], values='Values', title='No Thumbnail Present vs. Thumbnail Present')

# Show
thumbnail_pie.show()

### Quote URL Indicator

In [50]:
# Counter
qurl_counter = Counter(df.qurl_indicator)

# Lists for dict
qurl_viz = {'Indicator': qurl_counter.keys(),
            'Values': qurl_counter.values()}

# Pie chart
qurl_pie = px.pie(qurl_viz, names=['No Quote URL Present', 'Quote URL Present'], values='Values', title='No Quote URL Present vs. Quote URL Present')

# Show
qurl_pie.show()

### Conversations

For now, conversation lengths will be investigated.

In [51]:
# Description of size
df.conversation_size.describe()

count    2540.000000
mean        1.748819
std         2.573974
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        20.000000
Name: conversation_size, dtype: float64

In [52]:
# Exploration of the following:
# df['conversation_category'] 
# df['conversation_size'] 

# Conversation category counter
conv_cat_counter = Counter(df.conversation_category)

# Lists to dict
conv_cat_viz = {'Indicators': conv_cat_counter.keys(),
                'Values': conv_cat_counter.values()}

# Pie chart
conv_cat_pie = px.pie(conv_cat_viz, names=['Single Tweet', 'Multiple Tweets'], values='Values', title='Conversation Categories: Single Tweet vs. Multiple Tweets')

# Show
conv_cat_pie.show()

In [53]:
# Conversation size histogram
conv_hist = px.histogram(df, x='conversation_size', text_auto=True, title='Conversation Sizes')

# Show
conv_hist.show()

## Tweet Preprocessing
​
There are some things to do before tokenizing the tweets, such as translating emoji and expanding contractions. For this first research pass, the emoji will be treated in two steps:
1. Compile a list of emoji per tweet - this will enable analysis of the emoji as their own topic.
2. Replace emoji within the tweet with their string value - depending on your analysis, removing emoji may be more appropriate than substituting. I suspect substituting them will create a more insightful analysis, so I will not be removing them.
​
These steps will require two separate functions.

In [54]:
# Pulls out a list of emoji per tweet - for later analysis
def gatherEmoji(tweet):
    # Create a list
    emoji_list = demoji.findall_list(tweet)
    # Return that list
    return emoji_list

# Remove emoji
def dropEmoji(tweet):
    # Return the expanded string
    return demoji.replace(tweet)

In [55]:
# Gather the list of emojis per tweet
full_emoji_list = [gatherEmoji(t) for t in df.tweet]

# Check
print(f"There are {len(full_emoji_list)} items in the full emoji list for further analysis.")

There are 2540 items in the full emoji list for further analysis.


In [56]:
# List of tweets
df['no_emoji_tweets'] = [dropEmoji(t) for t in df.tweet]

Now, the tweets can be cleaned. This will:
- Remove mentions
- Remove hashtags
- Remove URLs

Other cleaning tasks can be performed afterwards:
- Remove special characters, numbers, and punctuation
- Remove stopwords

In [57]:
# Function to remove mentions
def removeMentions(tweet):
    # Removes mentions
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    return tweet

In [58]:
# Function to remove hashtags
def removeHashtags(tweet):
    # Remove hashtags
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    return tweet

In [59]:
# Function to remove links - links can be http or bit.ly or pic.twitter
def removeLinks(tweet):
    # Remove HTTP links
    tweet = re.sub(r'http\S+', '', tweet)
    # Remove bit.ly
    tweet = re.sub(r'bit.ly/\S+', '', tweet)
    # Substitute empty char for link bodies
    tweet = tweet.strip('[link]')   
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

In [60]:
# Create a function that brings all together
def cleanTweet(tweet):
    # remove mentions
    tweet = removeMentions(tweet)
    # remove hashtags
    tweet = removeHashtags(tweet)
    # remove links
    tweet = removeLinks(tweet)
    # return cleaned tweet
    return tweet

In [61]:
# Cleaned tweets
df['cleaned_tweets'] = [cleanTweet(t) for t in df.no_emoji_tweets]

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   index                  2540 non-null   int64 
 1   place                  5 non-null      object
 2   tweet                  2540 non-null   object
 3   username               2540 non-null   object
 4   name                   2540 non-null   object
 5   day                    2540 non-null   int64 
 6   hour                   2540 non-null   int64 
 7   urls                   2540 non-null   object
 8   video                  2540 non-null   int64 
 9   nlikes                 2540 non-null   int64 
 10  nreplies               2540 non-null   int64 
 11  nretweets              2540 non-null   int64 
 12  search                 2540 non-null   object
 13  reply_to               2540 non-null   object
 14  hashtag_counts         2540 non-null   int64 
 15  cashtag_counts       

## Tweet Processing

In [62]:
# Initialize the smaller English language model
nlp = spacy.load("en_core_web_sm")

In [63]:
# Set up stopwords
stopwords = nlp.Defaults.stop_words

# Check
print(f"There are {len(stopwords)} words in the stopwords list.")

There are 326 words in the stopwords list.


In [64]:
# Analysis of the current pipeline
analysis = nlp.analyze_pipes(pretty=True)

# print
print(analysis)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

In [65]:
# Create a contractions expansion function
def expandContractions(tweet):
    # Will expand any contractions found
    expanded_tweet = contractions.fix(tweet)
    # Return
    return expanded_tweet

In [66]:
# Expanded tweets
df['expanded_tweet'] = [expandContractions(t) for t in df.cleaned_tweets]

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   index                  2540 non-null   int64 
 1   place                  5 non-null      object
 2   tweet                  2540 non-null   object
 3   username               2540 non-null   object
 4   name                   2540 non-null   object
 5   day                    2540 non-null   int64 
 6   hour                   2540 non-null   int64 
 7   urls                   2540 non-null   object
 8   video                  2540 non-null   int64 
 9   nlikes                 2540 non-null   int64 
 10  nreplies               2540 non-null   int64 
 11  nretweets              2540 non-null   int64 
 12  search                 2540 non-null   object
 13  reply_to               2540 non-null   object
 14  hashtag_counts         2540 non-null   int64 
 15  cashtag_counts       

For each tweet, some useful data to have would be:
- Number of sentences
- Words per sentence
- Number of words
- Character count

In [67]:
# Add sentencizer
nlp.add_pipe('sentencizer', first=True)

In [68]:
# List to hold token details
sentence_count = [] # Count of sentences
parts_of_speech = [] # Parts of speech per lemma
lemma = [] # Find lemma
lemma_string = [] # Make lemmas into a string

# Iterate through column of expanded tweets
for tweet in tqdm(df.expanded_tweet):
    # Create the spacy object
    doc = nlp(tweet)
    # Convert numbers to words
    doc._.numerize()
    # Add the sentence count
    sentence_count.append(len([s for s in doc.sents]))
    # Remove punctuation - note that this will not remove punctuation that is embedded within a word or that has no white space on one side.
    lemma.append([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and token.pos_ != 'PRON'])
    lemma_string.append(' '.join([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and token.pos_ != 'PRON']))
    # Parts of speech
    parts_of_speech.append([token.lemma_.lower(), token.pos_] for token in doc)

100%|██████████| 2540/2540 [00:28<00:00, 89.53it/s] 


In [69]:
# Add sentence counts, all_tokens, and clean_tokens to the dataframe
df['sentence_counts'] = sentence_count
df['lemmas'] = lemma
df['lemma_string'] = lemma_string

## Creating a Base Model

To judge the base model, the following intrinsic statistics will be used:
1. Perplexity
2. Topic coherence

In [70]:
# id2word dictionary
id2word = Dictionary(df.lemmas)

# Size
print(f"There are {len(id2word)} lemma in the dictionary.")

There are 6552 lemma in the dictionary.


In [71]:
# Filter out based on length
id2word.filter_extremes(no_below=3, no_above=0.99)

# Size
print(f"There are {len(id2word)} lemma in the dictionary.")

There are 1939 lemma in the dictionary.


In [72]:
# Create a corpus
corpus = [id2word.doc2bow(d) for d in df.lemmas]

In [73]:
# Create the base model
base = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

In [74]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base.print_topics()]

In [75]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [76]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
american indian native black people hispanic white alaska like amp

------ Topic 1 ------
american indian native national museum tribe amp alaska black care

------ Topic 2 ------
american indian white indigenous want native black african time people

------ Topic 3 ------
american indian native littlefeather sacheen sister indigenous lie identity activist

------ Topic 4 ------
american indian native indigenous people like culture amp tribe claim



In [77]:
# Compute Perplexity - a measure of model fit. Lower is better
base_perplexity = base.log_perplexity(corpus)
print(f"Perplexity: {base_perplexity}.") 

Perplexity: -6.656599980036621.


In [78]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=base, texts=df['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_lda_model_base}")

Coherence Score: 0.33631462990851835


In [79]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base, corpus, id2word)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.017331  0.003525       1        1  28.101136
0      0.061193 -0.009358       2        1  22.862854
3     -0.146186 -0.026355       3        1  19.924755
1      0.073531 -0.092667       4        1  14.592032
2      0.028793  0.124856       5        1  14.519222, topic_info=              Term        Freq       Total Category  logprob  loglift
6    littlefeather  119.000000  119.000000  Default  30.0000  30.0000
9          sacheen  113.000000  113.000000  Default  29.0000  29.0000
10          sister  112.000000  112.000000  Default  28.0000  28.0000
0         activist   68.000000   68.000000  Default  27.0000  27.0000
516       hispanic  115.000000  115.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
55            know   19.896648  117.789531   Topic5  -5.2786   0.1513
7           native   33.442899  746.865787   Topic5  -4.7593  -1.1763
319      americans   14.846268   58.181436   Topic5  -5.5714   0.5639
85           leave   12.843318   33.579735   Topic5  -5.7163   0.9686
279          think   13.308519   76.671299   Topic5  -5.6807   0.1786

[359 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.014598  activist
0         3  0.963467  activist
839       2  0.801897       adj
941       2  0.857135  adoption
844       2  0.938208     adult
...     ...       ...       ...
412       5  0.095965      york
836       1  0.812137     youth
836       2  0.108285     youth
836       3  0.054142     youth
836       5  0.054142     youth

[744 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 2, 3])

/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Hyperparameter Tuning for Topic Modeling

The hyperparameters for topic modeling are:
- Number of topics (n_components)
- Learning decay (learning_decay), a way to control the learning rate.

In [81]:
# Count vectorizer
vect = CountVectorizer()

# Vectorized data
data_vect = vect.fit_transform(df.lemma_string)

In [82]:
# Define the search parameters
params = {'n_components': [3, 4, 5, 7, 10],
          'learning_decay': [0.5, 0.7, 0.9]}

In [83]:
# Model initialization
lda = LatentDirichletAllocation()

In [85]:
# Grid search initialization
grid = GridSearchCV(lda, param_grid=params, cv=None, error_score='raise', verbose=1)

In [86]:
# Do the grid search
grid.fit(data_vect)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


GridSearchCV(error_score='raise', estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [3, 4, 5, 7, 10]},
             verbose=1)

In [90]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print(f"Best Model's Params: {grid.best_params_}")

# Log Likelihood Score
print(f"Best Log Likelihood Score: {grid.best_score_}")

# Perplexity
print(f"Model Perplexity: {best_lda_model.perplexity(data_vect)}")

Best Model's Params: {'learning_decay': 0.9, 'n_components': 3}
Best Log Likelihood Score: -62161.65827000721
Model Perplexity: 1891.6332605386704
